# Database Helper Examples

This notebook demonstrates the refreshed helpers in `ml.common.database`.
They mirror the usage documented in `docs/database/` and provide quick
recipes for pulling Guild Wars 2 trading data with absolute or relative
time windows.


In [ ]:
import os
import sys
from datetime import UTC, datetime
from pathlib import Path

REPO_ROOT = Path.cwd().resolve().parents[1]
if str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))

from ml.common import DatabaseClient, database  # noqa: E402


## Environment setup

1. Ensure a `.env` file (or environment) provides `DB_URL` with access to the
   production replica described in `docs/database/public_tables.md`.
2. Optionally set `NOTEBOOK_ITEM_ID` if you want to analyse a different item ID.
3. Run the cells below inside the managed `uv` environment (`uv run --project ml jupyter notebook`).


In [7]:
client = DatabaseClient.from_env()
ITEM_ID = int(os.getenv("NOTEBOOK_ITEM_ID", "19702"))
ITEM_ID


19702

## Prices: relative lookback

Fetch the last week of prices for the current item. The helper accepts
`last_days`, `last_hours`, or `last_minutes` and automatically normalises
results to UTC.


In [8]:
prices_last_week = database.get_prices(
    client,
    item_id=ITEM_ID,
    last_days=7,
    limit=300,
    order="DESC",
)
prices_last_week.head()


,id,item_id,whitelisted,buy_quantity,buy_unit_price,sell_quantity,sell_unit_price,fetched_at,created_at
0,733781530,19702,True,279073,175,695337,215,2025-11-13 13:46:28.396531+00:00,2025-11-13 13:46:28.396531+00:00
1,733755136,19702,True,279397,175,696133,213,2025-11-13 13:41:58.237780+00:00,2025-11-13 13:41:58.237780+00:00
2,733727542,19702,True,279889,177,696349,213,2025-11-13 13:36:01.577245+00:00,2025-11-13 13:36:01.577245+00:00
3,733701548,19702,True,279733,176,696116,214,2025-11-13 13:30:46.874544+00:00,2025-11-13 13:30:46.874544+00:00
4,733674154,19702,True,280118,178,696610,200,2025-11-13 13:21:22.480343+00:00,2025-11-13 13:21:22.480343+00:00


## Prices: explicit window

Supply timezone-aware start/end dates for a deterministic slice (mirrors
examples in `docs/database/prices_schema.md`).


In [9]:
custom_window_prices = database.get_prices(
    client,
    item_id=ITEM_ID,
    start_time=datetime(2025, 1, 1, tzinfo=UTC),
    end_time=datetime(2025, 1, 8, tzinfo=UTC),
    limit=500,
    order="DESC",
)
custom_window_prices.head()


,id,item_id,whitelisted,buy_quantity,buy_unit_price,sell_quantity,sell_unit_price,fetched_at,created_at


## BLTC history helpers

The BLTC table stores timestamps in seconds. The convenience wrapper handles
conversion and ordering automatically. Use `last_days` for quick pulls or
provide explicit `start_time` / `end_time` values.


In [10]:
bltc_last_three_days = database.get_bltc_history(
    client,
    item_id=ITEM_ID,
    last_days=3,
    limit=3000,
    order="DESC",
)
bltc_last_three_days.head()


,id,item_id,timestamp,sell_price,buy_price,supply,demand,sold,offers,bought,bids,created_at
0,46904649901,19702,2025-11-12 09:25:12+00:00,219,173,691766,281350,139,0,150,0,2025-11-12 09:29:14.578217+00:00
1,46904649900,19702,2025-11-12 09:21:07+00:00,189,173,691945,281500,0,5,0,957,2025-11-12 09:29:14.578217+00:00
2,46904649899,19702,2025-11-12 09:17:01+00:00,190,172,691852,280543,0,4,0,307,2025-11-12 09:29:14.578217+00:00
3,46904649898,19702,2025-11-12 09:12:43+00:00,190,171,691861,280236,422,0,76,0,2025-11-12 09:29:14.578217+00:00
4,46904649897,19702,2025-11-12 09:08:53+00:00,220,171,692292,280312,284,0,25,0,2025-11-12 09:29:14.578217+00:00


In [11]:
bltc_custom_range = database.get_bltc_history(
    client,
    item_id=ITEM_ID,
    start_time=datetime(2025, 2, 1, tzinfo=UTC),
    end_time=datetime(2025, 2, 7, tzinfo=UTC),
    limit=5000,
    order="DESC",
)
bltc_custom_range.head()


,id,item_id,timestamp,sell_price,buy_price,supply,demand,sold,offers,bought,bids,created_at
0,165199915,19702,2025-02-07 00:00:00+00:00,181,158,533962,194860,13552,13715,7847,10062,2025-07-29 21:19:17.132938+00:00
1,165199914,19702,2025-02-06 18:00:00+00:00,186,158,535824,192752,10504,11749,7311,6185,2025-07-29 21:19:17.132938+00:00
2,165199913,19702,2025-02-06 12:00:00+00:00,187,169,536613,193129,8660,14978,6891,11243,2025-07-29 21:19:17.132938+00:00
3,165199912,19702,2025-02-06 06:00:00+00:00,180,155,528802,188777,9636,14471,9251,5322,2025-07-29 21:19:17.132938+00:00
4,165199911,19702,2025-02-06 00:00:00+00:00,191,164,524599,192706,15497,8918,11540,15129,2025-07-29 21:19:17.132938+00:00


## TP history helpers

`get_tp_history` mirrors the timed access patterns but honours the millisecond
precision used in `gw2tp_historical_prices`.


In [12]:
tp_last_hours = database.get_tp_history(
    client,
    item_id=ITEM_ID,
    last_hours=12,
    limit=4000,
    order="DESC",
)
tp_last_hours.head()


,id,item_id,timestamp,sell_price,buy_price,supply,demand,created_at
0,46143460659,19702,2025-11-13 12:24:00+00:00,212,174,697333,278461,2025-11-13 13:08:38.643457+00:00
1,46143460658,19702,2025-11-13 12:19:03+00:00,212,174,697460,278659,2025-11-13 13:08:38.643457+00:00
2,46143460657,19702,2025-11-13 12:14:07+00:00,212,174,697460,278736,2025-11-13 13:08:38.643457+00:00
3,46143460656,19702,2025-11-13 12:09:13+00:00,210,174,697730,277774,2025-11-13 13:08:38.643457+00:00
4,46143460655,19702,2025-11-13 12:04:20+00:00,175,174,697755,278463,2025-11-13 13:08:38.643457+00:00


In [13]:
tp_custom_range = database.get_tp_history(
    client,
    item_id=ITEM_ID,
    start_time=datetime(2025, 3, 1, tzinfo=UTC),
    end_time=datetime(2025, 3, 5, tzinfo=UTC),
    limit=5000,
    order="DESC",
)
tp_custom_range.head()


,id,item_id,timestamp,sell_price,buy_price,supply,demand,created_at
0,124190883,19702,2025-03-04 08:33:53+00:00,208,174,542344,218288,2025-07-29 18:51:00.335576+00:00
1,124190882,19702,2025-03-04 02:49:15+00:00,202,169,546722,213411,2025-07-29 18:51:00.335576+00:00
2,124190881,19702,2025-03-03 08:18:27+00:00,204,157,545060,201775,2025-07-29 18:51:00.335576+00:00
3,124190880,19702,2025-03-02 19:43:10+00:00,200,162,547210,206938,2025-07-29 18:51:00.335576+00:00
4,124190879,19702,2025-03-02 08:58:34+00:00,198,161,555564,210778,2025-07-29 18:51:00.335576+00:00


## Optional exports

Everything returned is a `pandas.DataFrame`. Use your preferred persistence or
analysis pattern—e.g. writing to `data/tests/` for reproducible snapshots.


In [14]:
export_dir = Path("../data/tests").resolve()
export_dir.mkdir(parents=True, exist_ok=True)
(prices_last_week
 .to_csv(export_dir / "notebook_prices_last_week.csv", index=False))
